In [ ]:
# Link dos dataframes: 
# Netflix: https://drive.google.com/file/d/1_ybq7a4dghc-TgaalA4IwE84ytGvJjpf/view?usp=sharing
# Votos: https://drive.google.com/file/d/1raW197S-64CaqgFvlk5OhIxocOMi7GMq/view?usp=sharing

import numpy as np 
import pandas as pd 


data_filmes = pd.read_csv('/content/drive/MyDrive/movies.csv', encoding="utf-8")
data_filmes.columns = ['show_id', 'title', 'listed_in']

data_filmes = data_filmes.set_index('show_id')


#----------------------------------------------------

data_votos = pd.read_csv('/content/drive/MyDrive/ratings.csv', encoding="utf-8")
data_votos.columns = ['user_id', 'show_id', 'rating', 'number']

#----------------------------------------------------

data_filmes['ratings'] = data_votos['show_id'].value_counts()
data_filmes['mean_rat'] = data_votos.groupby('show_id').mean()['rating']

#----------------------------------------------------

def user_rat_list(user_id):
  
  data_tmp = data_votos.query("user_id == %d" % user_id)
  
  return data_tmp[['show_id', 'rating']].set_index('show_id')


def user_rat_to(user_from, user_to):
  ratings_from = user_rat_list(user_from)
  ratings_to = user_rat_list(user_to)

  return ratings_from.join(ratings_to, 
                           lsuffix = "_from", rsuffix = "_to").dropna()

#-----------------------------------------------------

def dist_users(user_from, user_to, min = 5):
  ratings_from = user_rat_list(user_from)
  ratings_to = user_rat_list(user_to)

  data_tmp = ratings_from.join(ratings_to, 
             lsuffix = "_from", rsuffix = "_to").dropna()

  if (len(data_tmp) < min):

    return [user_from, user_to, None]
  
  return [user_from, user_to,  
          np.linalg.norm(data_tmp['ratings_from'] - 
                         data_tmp['ratings_to'])]

#-----------------------------------------------------

def dist_all(user_from):
  dist_list = []
  users = data_votos['user_id'].unique()

  for user_to in users:
    dist_list.append(dist_users(user_from, user_to))

  return pd.DataFrame(dist_list,
                      columns = ['from', 'to', 'distance']).dropna()
  
#-----------------------------------------------------

def nextmost_from(user_base):
  dists = dist_all(user_base)
  dists = dists.sort_values('distance')
  return dists.set_index('to').drop(user_base)

#-----------------------------------------------------

# Sugestão de filmes

def suggest_to(user_suggest):
  user_base_rat = user_rat_list(user_suggest)
  user_base_mov = user_base_rat.index

  similar_users = nextmost_from(user_suggest)
  similar = similar_users.iloc[0].name

  similar_rat = user_rat_list(simila)
  similar_rat = similar_rat.drop(user_base_mov, errors = 'ignore')

  reccs = similar_rat.sort_values('rating', ascending = False)

  return reccs.join(data_filmes)


#-----------------------------------------------------

s = input("Digite um código de usuário de 1 a 33 para receber sugestões: ")

suggest_to(int(s))